## explain(True)

- Logical Plan : 무엇을 할것인가 ? 추상적 계획
- Pysical Plan : 어떻게 할것인가? 구체적 실행 전략

1. 분석

- Unresolved Logical plan : 코드를 처음 파싱, 테이블이름 , 컬럼 모른다
- Catalog : 데이터저장소를 찾아서 컬럼명과 타입확인

2. 최적화

- Optimized Logical Plan : 비효율적인 로직을 똑똑하게 바꾼다

3. Pysical Plan

- 논리적 계획을 바탕으로 어떻게 실행할지 전략을 짠다

In [1]:
from pyspark.sql import (
    Row,
    SparkSession)
import pyspark.sql.functions as F

In [2]:
spark=(
    SparkSession
    .builder
    .appName("spark-explain")
    .master("spark://spark-master:7077")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/02 04:58:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data = [
    (1, "Alice", 25),
    (2, "Bob", 35),
    (3, "Charlie", 40)
]

columns = ["id", "name", "age"]

df = spark.createDataFrame(data, columns)


In [4]:
df.createOrReplaceTempView("employees")

In [5]:
query_df = spark.sql("""
    SELECT name
    FROM employees
    WHERE age > 30
""")

In [6]:
query_df.explain(True)

== Parsed Logical Plan ==
'Project ['name]
+- 'Filter ('age > 30)
   +- 'UnresolvedRelation [employees], [], false

== Analyzed Logical Plan ==
name: string
Project [name#1]
+- Filter (age#2L > cast(30 as bigint))
   +- SubqueryAlias employees
      +- View (`employees`, [id#0L,name#1,age#2L])
         +- LogicalRDD [id#0L, name#1, age#2L], false

== Optimized Logical Plan ==
Project [name#1]
+- Filter (isnotnull(age#2L) AND (age#2L > 30))
   +- LogicalRDD [id#0L, name#1, age#2L], false

== Physical Plan ==
*(1) Project [name#1]
+- *(1) Filter (isnotnull(age#2L) AND (age#2L > 30))
   +- *(1) Scan ExistingRDD[id#0L,name#1,age#2L]



In [7]:
query_df.show()

+-------+
|   name|
+-------+
|    Bob|
|Charlie|
+-------+



In [8]:
spark.stop()